# The Team

**Team Name**

[146] hey girl lets ML n chill

**Member, StudNr**

Sinan Maric, 529195

Joakim Ness, 544839

Ole Lasson, 543928

# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor
from datetime import datetime

In [2]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

raw_b = train_b

X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

# Preprocessing

In [3]:
def preprocessing_X(df):
    data = df.copy()
    if 'date_calc' in data.columns:
        data.drop('date_calc', axis=1, inplace=True)

    if 'date_forecast' in data.columns:
        data.set_index('date_forecast', inplace=True)

    
    data = data.groupby(pd.Grouper(freq='1H')).mean()
    
    
    data.dropna(how='all', inplace=True)
    
    data['hour'] = data.index.hour
    data['week'] = pd.to_numeric(data.index.isocalendar().week)
    data['week_hour'] = data['week'] * 100 + data['hour']
    data['month'] = data.index.month


    data['hour_sinus'] = np.sin(2*np.pi*data['hour']/ 23)
    data['hour_cosine'] = np.cos(2*np.pi*data['hour'] / 23)

    data['month_sinus'] = np.sin(2*np.pi*data['month'] / 12)
    data['month_cosine'] = np.cos(2*np.pi*data['month'] / 12)

    

    data.drop(columns=['snow_density:kgm3', 'month', 'hour', 'week','week_hour'], inplace=True)
    
    data.rename_axis('time', inplace=True)

    mapping = elevation_mapping = {
            6.0 : 'A',
            7.0 : 'B',
            24.0 : 'C'
        }
    
    data['location'] = data['elevation:m'].map(mapping)
    data['location'] = data['location'].astype(str)
    data.drop(columns=['elevation:m'], inplace=True)
    
    return data

def remove_constants_Y(df):
    data = df.copy()

    # Create a mask to identify sequences of 3 or more consecutive rows with the same 'pv_measurement' (non-zero)
    consecutive_mask = (data['pv_measurement'] == data['pv_measurement'].shift(1)) & (data['pv_measurement'] != 0)
    consecutive_count = consecutive_mask.groupby((~consecutive_mask).cumsum()).cumsum()
    mask_consecutive = (consecutive_count <= 3) | (~consecutive_mask)

    #create a mask to identify sequences of 24 or more consecutive 0-valued pv-measurement
    consecutive_mask_0 = (data['pv_measurement'] == data['pv_measurement'].shift(1)) & (data['pv_measurement'] == 0)
    consecutive_count_0 = consecutive_mask_0.groupby((~consecutive_mask_0).cumsum()).cumsum()
    mask_consecutive_0 = (consecutive_count_0 <= 24) | (~consecutive_mask_0)

    combined_mask = mask_consecutive & mask_consecutive_0

    filtered_data = data[combined_mask]

    return filtered_data


def preprocessing_Y(df):
    data = df.copy()
    if 'time' in data.columns:
        data.set_index('time', inplace=True)
    data.dropna(inplace=True)
    
    return data


def add_estimated_flag(df, estimated):
    data = df.copy()

    data['estimated'] = 'E' if estimated else 'O'

    return data

In [4]:
#combining observed and estimated data
observed_a = preprocessing_X(X_train_observed_a)
observed_a = add_estimated_flag(observed_a, False)

estimated_a = preprocessing_X(X_train_estimated_a)
estimated_a = add_estimated_flag(estimated_a, True)

x_train_a = pd.concat([observed_a, estimated_a], axis=0)
y_train_a = preprocessing_Y(train_a)
m_train_a = x_train_a.merge(y_train_a, how='inner', on='time')

observed_b = preprocessing_X(X_train_observed_b)
observed_b = add_estimated_flag(observed_b, False)

estimated_b = preprocessing_X(X_train_estimated_b)
estimated_b = add_estimated_flag(estimated_b, True)

x_train_b = pd.concat([observed_b, estimated_b], axis=0)
y_train_b = preprocessing_Y(train_b)
y_train_b = remove_constants_Y(y_train_b)
m_train_b = x_train_b.merge(y_train_b, how='inner', on='time')

observed_c = preprocessing_X(X_train_observed_c)
observed_c = add_estimated_flag(observed_c, False)

estimated_c = preprocessing_X(X_train_estimated_c)
estimated_c = add_estimated_flag(estimated_c, True)

x_train_c = pd.concat([observed_c, estimated_c], axis=0)
y_train_c = preprocessing_Y(train_c)
y_train_c = remove_constants_Y(y_train_c)
m_train_c = x_train_c.merge(y_train_c, how='inner', on='time')

x_train = pd.concat([m_train_a, m_train_b, m_train_c], axis=0, ignore_index=True)

y_train = x_train['pv_measurement']

x_train = x_train.drop(columns=['pv_measurement'], axis=1)

X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=10)

X_train_a = X_train[X_train['location'] == 'A']
X_train_b = X_train[X_train['location'] == 'B']
X_train_c = X_train[X_train['location'] == 'C']

Y_train_a = Y_train[X_train['location'] == 'A']
Y_train_b = Y_train[X_train['location'] == 'B']
Y_train_c = Y_train[X_train['location'] == 'C']

X_val_a = X_val[X_val['location'] == 'A']
X_val_b = X_val[X_val['location'] == 'B']
X_val_c = X_val[X_val['location'] == 'C']

Y_val_a = Y_val[X_val['location'] == 'A']
Y_val_b = Y_val[X_val['location'] == 'B']
Y_val_c = Y_val[X_val['location'] == 'C']

x_test_a = preprocessing_X(X_test_estimated_a)
x_test_a = add_estimated_flag(x_test_a, True)

x_test_b = preprocessing_X(X_test_estimated_b)
x_test_b = add_estimated_flag(x_test_b, True)

x_test_c = preprocessing_X(X_test_estimated_c)
x_test_c = add_estimated_flag(x_test_c, True)




x_test = pd.concat([x_test_a, x_test_b, x_test_c], axis=0, ignore_index=True)

NameError: name 'MinMaxScaler' is not defined

# AutoGluon

In [ ]:
X_train_a = X_train_a.copy()
X_val_a = X_val_a.copy()

X_train_a['pv_measurement'] = Y_train_a
X_val_a['pv_measurement'] = Y_val_a
predictor_a = TabularPredictor(
                label='pv_measurement',
                eval_metric='mean_absolute_error').fit(
                    train_data=X_train_a,
                    tuning_data=X_val_a
                    )

No path specified. Models will be saved in: "AutogluonModels/ag-20231109_190847/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231109_190847/"
AutoGluon Version:  0.8.2
Python Version:     3.9.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.6.0: Wed Jul  5 22:22:05 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6000
Disk Space Avail:   630.56 GB / 994.66 GB (63.4%)
Train Data Rows:    27220
Train Data Columns: 49
Tuning Data Rows:    6841
Tuning Data Columns: 49
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 625.33327, 1156.81574)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary',

[1000]	valid_set's l1: 170.153
[2000]	valid_set's l1: 163.496
[3000]	valid_set's l1: 160.526
[4000]	valid_set's l1: 158.519
[5000]	valid_set's l1: 157.406
[6000]	valid_set's l1: 156.537
[7000]	valid_set's l1: 155.928
[8000]	valid_set's l1: 155.453
[9000]	valid_set's l1: 155.163
[10000]	valid_set's l1: 154.928


	-154.9279	 = Validation score   (-mean_absolute_error)
	34.47s	 = Training   runtime
	0.77s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 171.235
[2000]	valid_set's l1: 168.975
[3000]	valid_set's l1: 167.793
[4000]	valid_set's l1: 167.32
[5000]	valid_set's l1: 167.111
[6000]	valid_set's l1: 166.949
[7000]	valid_set's l1: 166.833
[8000]	valid_set's l1: 166.703
[9000]	valid_set's l1: 166.666
[10000]	valid_set's l1: 166.632


	-166.6311	 = Validation score   (-mean_absolute_error)
	27.47s	 = Training   runtime
	0.79s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-184.8604	 = Validation score   (-mean_absolute_error)
	14.21s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ...
	-172.8182	 = Validation score   (-mean_absolute_error)
	92.58s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-184.0486	 = Validation score   (-mean_absolute_error)
	2.76s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-181.0095	 = Validation score   (-mean_absolute_error)
	19.0s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: XGBoost ...
	-177.6836	 = Validation score   (-mean_absolute_error)
	86.82s	 = Training   runtime
	0.78s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-168.9959	 = Validation score   (-mean_absolute_error)
	30.19s	 = Training   runtime
	0.03s	 = Validation runtime
Fitti

[1000]	valid_set's l1: 165.067
[2000]	valid_set's l1: 163.675
[3000]	valid_set's l1: 163.349
[4000]	valid_set's l1: 163.279
[5000]	valid_set's l1: 163.255
[6000]	valid_set's l1: 163.246
[7000]	valid_set's l1: 163.24
[8000]	valid_set's l1: 163.239
[9000]	valid_set's l1: 163.238
[10000]	valid_set's l1: 163.238


	-163.2376	 = Validation score   (-mean_absolute_error)
	96.31s	 = Training   runtime
	2.07s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-151.6939	 = Validation score   (-mean_absolute_error)
	0.15s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 411.4s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20231109_190847/")


In [ ]:
X_train_b = X_train_b.copy()
X_val_b = X_val_b.copy()

X_train_b['pv_measurement'] = Y_train_b
X_val_b['pv_measurement'] = Y_val_b
predictor_b = TabularPredictor(
                label='pv_measurement',
                eval_metric='mean_absolute_error').fit(
                    train_data=X_train_b,
                    tuning_data=X_val_b
                    )

No path specified. Models will be saved in: "AutogluonModels/ag-20231109_191539/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231109_191539/"
AutoGluon Version:  0.8.2
Python Version:     3.9.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.6.0: Wed Jul  5 22:22:05 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6000
Disk Space Avail:   629.62 GB / 994.66 GB (63.3%)
Train Data Rows:    21539
Train Data Columns: 49
Tuning Data Rows:    5311
Tuning Data Columns: 49
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1152.3, 0.0, 103.28401, 209.28429)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', '

[1000]	valid_set's l1: 24.2459
[2000]	valid_set's l1: 23.4345
[3000]	valid_set's l1: 23.0482
[4000]	valid_set's l1: 22.8585
[5000]	valid_set's l1: 22.6823
[6000]	valid_set's l1: 22.5591
[7000]	valid_set's l1: 22.4742
[8000]	valid_set's l1: 22.3831
[9000]	valid_set's l1: 22.3294
[10000]	valid_set's l1: 22.2842


	-22.2833	 = Validation score   (-mean_absolute_error)
	26.31s	 = Training   runtime
	0.51s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 25.2685
[2000]	valid_set's l1: 24.9041
[3000]	valid_set's l1: 24.7856
[4000]	valid_set's l1: 24.7308
[5000]	valid_set's l1: 24.7098
[6000]	valid_set's l1: 24.6947
[7000]	valid_set's l1: 24.6884
[8000]	valid_set's l1: 24.6797
[9000]	valid_set's l1: 24.675
[10000]	valid_set's l1: 24.6724


	-24.6724	 = Validation score   (-mean_absolute_error)
	25.47s	 = Training   runtime
	0.67s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-27.3972	 = Validation score   (-mean_absolute_error)
	13.34s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost ...
	-25.0711	 = Validation score   (-mean_absolute_error)
	89.68s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-27.2145	 = Validation score   (-mean_absolute_error)
	2.11s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-27.1468	 = Validation score   (-mean_absolute_error)
	11.98s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-25.6187	 = Validation score   (-mean_absolute_error)
	65.12s	 = Training   runtime
	0.61s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-25.1312	 = Validation score   (-mean_absolute_error)
	23.38s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting mod

[1000]	valid_set's l1: 24.2838
[2000]	valid_set's l1: 24.1549
[3000]	valid_set's l1: 24.1251
[4000]	valid_set's l1: 24.1174
[5000]	valid_set's l1: 24.1152
[6000]	valid_set's l1: 24.1145
[7000]	valid_set's l1: 24.1143
[8000]	valid_set's l1: 24.1142
[9000]	valid_set's l1: 24.1142
[10000]	valid_set's l1: 24.1142


	-24.1142	 = Validation score   (-mean_absolute_error)
	93.71s	 = Training   runtime
	1.92s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-21.9789	 = Validation score   (-mean_absolute_error)
	0.21s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 357.43s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20231109_191539/")


In [ ]:
X_train_c = X_train_c.copy()
X_val_c = X_val_c.copy()

X_train_c['pv_measurement'] = Y_train_c
X_val_c['pv_measurement'] = Y_val_c
predictor_c = TabularPredictor(
                label='pv_measurement',
                eval_metric='mean_absolute_error').fit(
                    train_data=X_train_c,
                    tuning_data=X_val_c
                    )

No path specified. Models will be saved in: "AutogluonModels/ag-20231109_192136/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231109_192136/"
AutoGluon Version:  0.8.2
Python Version:     3.9.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.6.0: Wed Jul  5 22:22:05 PDT 2023; root:xnu-8796.141.3~6/RELEASE_ARM64_T6000
Disk Space Avail:   628.87 GB / 994.66 GB (63.2%)
Train Data Rows:    17537
Train Data Columns: 49
Tuning Data Rows:    4422
Tuning Data Columns: 49
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (999.6, 0.0, 91.46277, 176.08288)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary'

[1000]	valid_set's l1: 22.3454
[2000]	valid_set's l1: 21.4621
[3000]	valid_set's l1: 21.1152
[4000]	valid_set's l1: 20.905
[5000]	valid_set's l1: 20.7505
[6000]	valid_set's l1: 20.6662
[7000]	valid_set's l1: 20.6022
[8000]	valid_set's l1: 20.5594
[9000]	valid_set's l1: 20.5253
[10000]	valid_set's l1: 20.5022


	-20.5021	 = Validation score   (-mean_absolute_error)
	26.76s	 = Training   runtime
	0.45s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 22.6198
[2000]	valid_set's l1: 22.2574
[3000]	valid_set's l1: 22.1781
[4000]	valid_set's l1: 22.136
[5000]	valid_set's l1: 22.1093
[6000]	valid_set's l1: 22.0997
[7000]	valid_set's l1: 22.0908
[8000]	valid_set's l1: 22.0863
[9000]	valid_set's l1: 22.084
[10000]	valid_set's l1: 22.0823


	-22.0821	 = Validation score   (-mean_absolute_error)
	22.9s	 = Training   runtime
	0.55s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-24.8533	 = Validation score   (-mean_absolute_error)
	7.92s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: CatBoost ...
	-22.6419	 = Validation score   (-mean_absolute_error)
	87.1s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-24.4147	 = Validation score   (-mean_absolute_error)
	1.61s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-23.9851	 = Validation score   (-mean_absolute_error)
	9.73s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-23.2409	 = Validation score   (-mean_absolute_error)
	61.29s	 = Training   runtime
	0.44s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-21.8483	 = Validation score   (-mean_absolute_error)
	28.79s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: 

[1000]	valid_set's l1: 22.0287
[2000]	valid_set's l1: 21.9013
[3000]	valid_set's l1: 21.8847
[4000]	valid_set's l1: 21.8815
[5000]	valid_set's l1: 21.881
[6000]	valid_set's l1: 21.8809
[7000]	valid_set's l1: 21.8809
[8000]	valid_set's l1: 21.8809
[9000]	valid_set's l1: 21.8809
[10000]	valid_set's l1: 21.8809


	-21.8809	 = Validation score   (-mean_absolute_error)
	84.94s	 = Training   runtime
	1.4s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-19.9105	 = Validation score   (-mean_absolute_error)
	0.13s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 336.1s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20231109_192136/")


In [ ]:
y_pred_a = predictor_a.predict(x_test_a)
y_pred_b = predictor_b.predict(x_test_b)
y_pred_c = predictor_c.predict(x_test_c)

y_pred_c.head()

time
2023-05-01 00:00:00   -1.074589
2023-05-01 01:00:00   -1.661733
2023-05-01 02:00:00   -1.057483
2023-05-01 03:00:00   -1.326593
2023-05-01 04:00:00   -1.053501
Name: pv_measurement, dtype: float32

In [ ]:
perf = predictor_a.evaluate(X_val_a, silent=True)
predictor_a.leaderboard(X_val_a, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-151.693877,-151.693877,3.742468,3.655949,188.590167,0.009713,0.000789,0.147905,2,True,10
1,LightGBMXT,-154.927880,-154.927880,0.682783,0.768952,34.474560,0.682783,0.768952,34.474560,1,True,1
2,LightGBMLarge,-163.237628,-163.237628,2.196334,2.071631,96.312754,2.196334,2.071631,96.312754,1,True,9
3,LightGBM,-166.631062,-166.631062,0.816492,0.787166,27.465005,0.816492,0.787166,27.465005,1,True,2
4,NeuralNetTorch,-168.995947,-168.995947,0.037146,0.027411,30.189943,0.037146,0.027411,30.189943,1,True,8
5,CatBoost,-172.818230,-172.818230,0.052380,0.014939,92.584997,0.052380,0.014939,92.584997,1,True,4
6,XGBoost,-177.683620,-177.683620,0.897521,0.779569,86.821126,0.897521,0.779569,86.821126,1,True,7
7,NeuralNetFastAI,-181.009512,-181.009512,0.065075,0.046905,19.000463,0.065075,0.046905,19.000463,1,True,6
8,ExtraTreesMSE,-184.048648,-184.048648,0.218256,0.047898,2.762653,0.218256,0.047898,2.762653,1,True,5
9,RandomForestMSE,-184.860396,-184.860396,0.249706,0.045791,14.213299,0.249706,0.045791,14.213299,1,True,3


In [ ]:
autogluon_predictions = pd.concat([y_pred_a, y_pred_b, y_pred_c], axis=0, ignore_index=True).to_frame()
autogluon_predictions.index.name = 'id'
autogluon_predictions.rename(columns={'pv_measurement': 'prediction'}, inplace=True)

# Catboost

In [ ]:
from catboost import CatBoostRegressor
from catboost import Pool

params = {
    'depth' : 9,
    'iterations' : 1000,
    'loss_function' : 'MAE'
}

def catboost(X_train, Y_train, X_val, Y_val):
    
    model = CatBoostRegressor(**params)

    train_pool = Pool(data=X_train, label=Y_train, cat_features=['location', 'estimated'])

    eval_pool = Pool(data=X_val, label=Y_val, cat_features=['location', 'estimated'])

    model.fit(train_pool, use_best_model=True, eval_set=eval_pool)
    return model 



def train_predict(single=False):
    lenlist = []
    if not single:

        model = catboost(X_train, Y_train, X_val, Y_val)
        
        predictions = pd.DataFrame(model.predict(x_test), columns=['prediction'])

        predictions.index.name = 'id'

        return predictions

    else:
        model_a = catboost(X_train_a, Y_train_a, X_val_a, Y_val_a)
        predict_a = model_a.predict(x_test_a)
        predictions_a = pd.DataFrame(predict_a, columns=['prediction'])

        model_b = catboost(X_train_b, Y_train_b, X_val_b, Y_val_b)
        predict_b = model_b.predict(x_test_b)
        predictions_b = pd.DataFrame(predict_b, columns=['prediction'])
        
        model_c = catboost(X_train_c, Y_train_c, X_val_c, Y_val_c)
        predict_c = model_c.predict(x_test_c)
        predictions_c = pd.DataFrame(predict_c, columns=['prediction'])

        predictions = pd.concat([predictions_a, predictions_b, predictions_c], axis=0, ignore_index=True)

        predictions.index.name = 'id'

        return predictions



def transformed_train_predict(single=False):
    if not single:

        model = catboost(transformed_X_train, transformed_Y_train, transformed_X_val, transformed_Y_val)
        
        predictions = pd.DataFrame(model.predict(x_test), columns=['prediction'])

        predictions.index.name = 'id'

        return predictions

    else:
        model_a = catboost(transformed_X_train_a, transformed_Y_train_a, transformed_X_val_a, transformed_Y_val_a)
        predict_a = model_a.predict(x_test_a)
        predictions_a = pd.DataFrame(predict_a, columns=['prediction'])
        predictions_a.loc[:, 'prediction'] = scaler_a.inverse_transform(predictions_a['prediction'].values.reshape(-1, 1))
        

        model_b = catboost(transformed_X_train_b, transformed_Y_train_b, transformed_X_val_b, transformed_Y_val_b)
        predict_b = model_b.predict(x_test_b)
        predictions_b = pd.DataFrame(predict_b, columns=['prediction'])
        predictions_b.loc[:, 'prediction'] = scaler_b.inverse_transform(predictions_b['prediction'].values.reshape(-1, 1))

        
        model_c = catboost(transformed_X_train_c, transformed_Y_train_c, transformed_X_val_c, transformed_Y_val_c)
        predict_c = model_c.predict(x_test_c)
        predictions_c = pd.DataFrame(predict_c, columns=['prediction'])
        predictions_c.loc[:, 'prediction'] = scaler_c.inverse_transform(predictions_c['prediction'].values.reshape(-1, 1))


        predictions = pd.concat([predictions_a, predictions_b, predictions_c], axis=0, ignore_index=True)


        predictions.index.name = 'id'
        

        return predictions

In [ ]:
# generating 100 different train_test splits
for i in range(100):
    
    #creating datasets based on seed state i
    X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=i)

    X_train_a = X_train[X_train['location'] == 'A']
    X_train_b = X_train[X_train['location'] == 'B']
    X_train_c = X_train[X_train['location'] == 'C']

    Y_train_a = Y_train[X_train['location'] == 'A']
    Y_train_b = Y_train[X_train['location'] == 'B']
    Y_train_c = Y_train[X_train['location'] == 'C']

    X_val_a = X_val[X_val['location'] == 'A']
    X_val_b = X_val[X_val['location'] == 'B']
    X_val_c = X_val[X_val['location'] == 'C']

    Y_val_a = Y_val[X_val['location'] == 'A']
    Y_val_b = Y_val[X_val['location'] == 'B']
    Y_val_c = Y_val[X_val['location'] == 'C']


    temp_catboost_prediction = train_predict(single=True)


    #creating master dataframe
    if i == 0:
        catboost_master = pd.DataFrame()

    #adding prediction to master dataframe
    catboost_master[f'prediction_{i}'] = temp_catboost_prediction['prediction']
        

#establishing final dataframe
catboost_predictions = pd.DataFrame()

#calculating average for printout
catboost_master['avg'] = catboost_master.mean(axis=1)

#calculating mean
catboost_predictions['prediction'] = catboost_master.mean(axis=1)

# Postprocessing

In [ ]:
def postprocessing(features_df, preds_df):
    features = features_df.copy()
    preds = preds_df.copy()

    # Set the index of 'features' to match 'preds'
    features['is_day:idx'].index = preds.index

    # Setting all night-time predictions to zero
    preds.loc[features['is_day:idx'] == 0, 'prediction'] = 0

    # Setting all low values to zero
    preds['prediction'] = preds['prediction'].apply(lambda x: 0.0 if x < 1 else x)

    return preds

In [ ]:
catboost_predictions_proc = postprocessing(x_test, catboost_predictions)
autogluon_predictions_proc = postprocessing(x_test, autogluon_predictions)

In [ ]:
final = pd.DataFrame()

final['prediction'] =(
                0.55 * catboost_predictions_proc['prediction'] + 
                0.45 * autogluon_predictions_proc['prediction']     
                )            

final.index.name = 'id'

now = datetime.now()
current_time = now.strftime("%H:%M:%S")

final.to_csv(f'pred_{current_time}.csv')